In [ ]:
from mailbox import mbox
from transformers import pipeline
from utils import get_entity_books, get_text_messages, write_entity_books, get_content_dataset


pipe = pipeline("token-classification", model="saattrupdan/nbailab-base-ner-scandi", aggregation_strategy='first')

box = mbox("epost/Sv_Taleopptak_fra_DSS.mbox", create=False)

: 

In [ ]:
from tqdm import tqdm

content_dataset = get_content_dataset(box=box)

exception_indices = []
try:
    res = [e for e in tqdm(pipe(content_dataset))]
except Exception as ex:
    print(ex)
    res = []
    for i, e in tqdm(enumerate(content_dataset), total=len(content_dataset)):
        try: 
            pipe(e)
            res.append(e)
        except Exception as ex:
            exception_indices.append(i)

if exception_indices:
    messages = get_text_messages(box)
    bad_messages = [messages[i] for i in exception_indices]
    for msg in bad_messages:
        content = msg._payload
        headers = dict(msg._headers)
        print(headers)
        

: 

In [ ]:
from pathlib import Path       

category_directory_map = {
    "PER": "Person",
    "LOC": "Place",
    "ORG": "Organisation",
    "MISC": "Other"
}

labels = {e.split("-")[1] for e in pipe.model.config.label2id if "-" in e}
labels

for e in labels:
    category_directory_map[e]

: 

In [ ]:
len([x for e in res for x in e])

: 

In [ ]:
entity_books = get_entity_books(Path("/home/tita/epadd-appraisal/user/data/sessions/EntityBooks"), sub_dirs=category_directory_map.values())
print([(k, len(v)) for k,v in entity_books.items()])
print([(k, len(set(v))) for k,v in entity_books.items()])


: 

In [ ]:
threshold = 0.8
ents = {(e["word"], e["entity_group"]) for ent_list in res for e in ent_list if e["score"] > threshold}
len(ents)

: 

In [ ]:
for name, ent_type in ents:
    entity_books[category_directory_map[ent_type]].append(name)
for k, v in entity_books.items():
    entity_books[k] = sorted(list(set(v)))

print([(k, len(v)) for k,v in entity_books.items()])
print([(k, len(set(v))) for k,v in entity_books.items()])



: 

In [ ]:
write_entity_books(entity_books=entity_books, parent_dir=Path("/home/tita/epadd-appraisal/user/data/sessions/EntityBooks"))

: 